In [80]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
# from PIL import Image
from IPython.display import display
from IPython.display import Image


In [81]:
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

In [82]:
import pandas as pd
import time

In [83]:
df = pd.read_csv('./../data/nyc_bar_crawl.csv')

In [84]:
# Yelp Fusion API

API_KEY= '4nRS21VgE79PQdVbjIz8wYPndzvh94hFffLad8WesROrkUBAqMftFyEUrcHlD1jLLBb6aggJD6NAEZcFOLjTn2hi3DGTM-jpiSisM8Aq4REo92Jwk6znbI-qKvAhY3Yx' 

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3


In [85]:

def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

#     print(u'{0} businesses found, querying business info ' \
#         'for the top result "{1}" ...'.format(
#             len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
#     print(response)
#     pprint.pprint(response, indent=2)
    
#     pprint.pprint(response, indent=2)
    
    return response




In [86]:
def process_hours(i,hours):
    
    
#     print("Type hours: ",type(hours))
    for h in hours:
#         print("Type h: ",type(h))
        for io in h:
#             print("Type io: ",type(io))
#             print("Key: ",io)
#             print("Value: ")
#             pprint.pprint(h[io])
            
            if io == 'open':
                for o in h[io]:
#                     print(o)
                    for key in o:
#                         print(key,":",o[key])
#                         print(key)
                        if key == 'day' and o[key] == 0:
                            print(df.loc[i,'bar_pub_name'])
#                             df.loc[i, 'day_0_hours_start']
                            df.loc[i,'day_0_hours_start'] = o['start']
                            df.loc[i,'day_0_hours_end'] = o['end']
                        elif key == 'day' and o[key] == 1:
#                         print(key,":",o[key])
                            df.loc[i,'day_1_hours_start'] = o['start']
                            df.loc[i,'day_1_hours_end'] = o['end']
                        elif key == 'day' and o[key] == 2:
#                         print(key,":",o[key])
                            df.loc[i,'day_2_hours_start'] = o['start']
                            df.loc[i,'day_2_hours_end'] = o['end']    
                        elif key == 'day' and o[key] == 3:
#                         print(key,":",o[key])
                            df.loc[i,'day_3_hours_start'] = o['start']
                            df.loc[i,'day_3_hours_end'] = o['end']    
                        elif key == 'day' and o[key] == 4:
#                         print(key,":",o[key])
                            df.loc[i,'day_4_hours_start'] = o['start']
                            df.loc[i,'day_4_hours_end'] = o['end']    
                        elif key == 'day' and o[key] == 5:
#                         print(key,":",o[key])
                            df.loc[i,'day_5_hours_start'] = o['start']
                            df.loc[i,'day_5_hours_end'] = o['end']    
                        elif key == 'day' and o[key] == 6:
#                         print(key,":",o[key])
                            df.loc[i,'day_6_hours_start'] = o['start']
                            df.loc[i,'day_6_hours_end'] = o['end']
#                         break
                break
            break
        break

In [90]:
def process_rating(i, rating):
#     print("i: ",i,"rating: ",rating)
    df.loc[i,'yelp_rating'] = rating

In [91]:
def process_price(i,price):
#     print("i: ",i,"price: ",price)
    df.loc[i,'yelp_price'] = price

In [92]:
def process_url(i,url):
    print("i: ",i,"url: ",url)
    df.loc[i,'yelp_url'] = url

In [93]:
def process_image_url(i,image_url):
#     print("i: ",i,"image_url: ",image_url)
#     with urllib.request.urlopen(image_url) as url:
#         with open('temp.jpg', 'wb') as f:
#             f.write(url.read())
#     img = Image.open('temp.jpg')
#     img.show()
#     ig=Image(image_url, width = 500)
#     display(ig)
    df.loc[i,'yelp_image_url'] = image_url

In [94]:
def main():
    parser = argparse.ArgumentParser()

#     parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
#                         type=str, help='Search term (default: %(default)s)')
#     parser.add_argument('-l', '--location', dest='location',
#                         default=DEFAULT_LOCATION, type=str,
#                         help='Search location (default: %(default)s)')
#     parser.add_argument('-lo', '--longitude', dest='longitude',
#                         type=str, help='Search longitude')
#     parser.add_argument('-la', '--latitude', dest='latitude',
#                         type=str, help='Search latitude')

#     input_values = parser.parse_args()

    location = 'New York'

    for i in range(len(df)):
        print(df['bar_pub_name'][i])
        term = df['bar_pub_name'][i]
    
        try:
            response = query_api(term, location)
    #         query_api(input_values.term, input_values.location)
            if response is not None:
                for res in response:
                        if res == 'hours':
                            process_hours(i,response[res])
                        if res == 'rating':
                            process_rating(i,response[res])
                        if res == 'price':
                            process_price(i,response[res])
                        if res == 'url':
                            process_url(i,response[res])
                        if res == 'image_url':
                            process_image_url(i,response[res])
        except HTTPError as error:
            sys.exit(
                'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
                    error.code,
                    error.url,
                    error.read(),
                )
            )
            
        
#                 print(df.head())
                


In [16]:
# if __name__ == '__main__':
#     main()

310 BOWERY BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/RpgITHQX66KTVk3bN0w6dw ...
Result for business "RpgITHQX66KTVk3bN0w6dw" found:
{ 'alias': '310-bowery-bar-new-york',
  'categories': [ {'alias': 'venues', 'title': 'Venues & Event Spaces'},
                  {'alias': 'lounges', 'title': 'Lounges'}],
  'coordinates': {'latitude': 40.7248881372237, 'longitude': -73.9925056790985},
  'display_phone': '(917) 409-3036',
  'hours': [ { 'hours_type': 'REGULAR',
               'is_open_now': False,
               'open': [ { 'day': 0,
                           'end': '0200',
                           'is_overnight': True,
                           'start': '1500'},
                         { 'day': 1,
                           'end': '0200',
                           'is_overnight': True,
                           'start': '1500'},
                         { 'day': 2,
                           'end': '0200',
               

In [95]:
if __name__ == '__main__':
    main()

310 BOWERY BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/RpgITHQX66KTVk3bN0w6dw ...
Result for business "RpgITHQX66KTVk3bN0w6dw" found:
i:  0 url:  https://www.yelp.com/biz/310-bowery-bar-new-york?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
310 BOWERY BAR
55 BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/9Ke2V1SJVOPyr4z3Bj4_IQ ...
Result for business "9Ke2V1SJVOPyr4z3Bj4_IQ" found:
i:  1 url:  https://www.yelp.com/biz/sky-55-bar-and-grill-new-york?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
55 BAR
69 BAR LOUNGE
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/bvba7OYmEvcO8kOEez35gw ...
Result for business "bvba7OYmEvcO8kOEez35gw" found:
i:  2 url:  http

Querying https://api.yelp.com/v3/businesses/KFnr0CGsHQ2ABFHbLNtobQ ...
Result for business "KFnr0CGsHQ2ABFHbLNtobQ" found:
i:  23 url:  https://www.yelp.com/biz/central-park-conservancy-new-york?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
FOREST PARK SNACK BAR
GARDEN PUB
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/tgF-xZljBh5Y9PxCm1NYwg ...
Result for business "tgF-xZljBh5Y9PxCm1NYwg" found:
i:  24 url:  https://www.yelp.com/biz/flynns-garden-inn-sunnyside?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
GARDEN PUB
HARBOR INN SPORTS BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/7Bb_IDWV_zJqpBjAvxuBIA ...
Result for business "7Bb_IDWV_zJqpBjAvxuBIA" found:
i:  25 url:  https://www.yelp.com/biz/coopers-craft-a

Querying https://api.yelp.com/v3/businesses/IGZkGbwE36iE1yaDW5NNKA ...
Result for business "IGZkGbwE36iE1yaDW5NNKA" found:
i:  45 url:  https://www.yelp.com/biz/the-flying-cock-new-york-2?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
THE GUTTER BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/Ct6ElbYDvXpQtYO_mkHUmw ...
Result for business "Ct6ElbYDvXpQtYO_mkHUmw" found:
i:  46 url:  https://www.yelp.com/biz/the-gutter-brooklyn?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
THE GUTTER BAR
THIRD AVENUE ALE HOUSE
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/xwdyPDcF1PifAiXyNnyA5Q ...
Result for business "xwdyPDcF1PifAiXyNnyA5Q" found:
i:  47 url:  https://www.yelp.com/biz/third-avenue-ale-house-new-york?adjust_crea

Querying https://api.yelp.com/v3/businesses/dYm7DXMy1r8kCmKiz1MO8w ...
Result for business "dYm7DXMy1r8kCmKiz1MO8w" found:
i:  65 url:  https://www.yelp.com/biz/crave-fishbar-new-york?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
CRAVE FISH BAR
DAIGO HAND ROLL BAR (DEKALB MARKET)
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/KxOAVoyJ55G-gEQqhnBpAQ ...
Result for business "KxOAVoyJ55G-gEQqhnBpAQ" found:
i:  66 url:  https://www.yelp.com/biz/daigo-handroll-bar-brooklyn?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
DAIGO HAND ROLL BAR (DEKALB MARKET)
DYLAN'S CANDY BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/LcqOUCT0rx9gRDs_KO5IIA ...
Result for business "LcqOUCT0rx9gRDs_KO5IIA" found:
i:  67 url:  https://www.

Querying https://api.yelp.com/v3/businesses/Zp-Fxv6lEvUrtLwkHlykUQ ...
Result for business "Zp-Fxv6lEvUrtLwkHlykUQ" found:
i:  85 url:  https://www.yelp.com/biz/millers-ale-house-paramus-5?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
MILLER'S ALE HOUSE
MIS ANTOJITOS AND NATURAL JUICE BAR
Querying https://api.yelp.com/v3/businesses/search ...
No businesses for MIS ANTOJITOS AND NATURAL JUICE BAR in New York found.
PARRILLADAS SUNRISE MEXICAN RESTAURANT BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/2hmhZim4bX2lbjPSNc7LaA ...
Result for business "2hmhZim4bX2lbjPSNc7LaA" found:
i:  87 url:  https://www.yelp.com/biz/javelina-new-york-7?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
PARRILLADAS SUNRISE MEXICAN RESTAURANT BAR
PC'S BAR AND GRILL
Querying https://api.ye

...

Querying https://api.yelp.com/v3/businesses/MNmfii9J3VXy1G5e-3ijqw ...
Result for business "MNmfii9J3VXy1G5e-3ijqw" found:
i:  647 url:  https://www.yelp.com/biz/fierce-grace-nyc-new-york?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
MOVE WITH GRACE YOGA AND JUICE BAR
MUG-Z'S SPORTS BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/B-mnfJNSVNx1t_eQpnF1bA ...
Result for business "B-mnfJNSVNx1t_eQpnF1bA" found:
i:  648 url:  https://www.yelp.com/biz/40-40-club-new-york?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
MUG-Z'S SPORTS BAR
NORDSTROM SHOE BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/iqnQnV3gUv-xSjxLiWqmww ...
Result for business "iqnQnV3gUv-xSjxLiWqmww" found:
i:  649 url:  https://www.yelp.com/biz/ds

Querying https://api.yelp.com/v3/businesses/xEVOWFEQhNriHci3g91M7g ...
Result for business "xEVOWFEQhNriHci3g91M7g" found:
i:  667 url:  https://www.yelp.com/biz/tilt-bk-brooklyn?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
TIME TEQUILA BAR CAFE
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/ZZSnFJU1iCjLssjJ93oMhg ...
Result for business "ZZSnFJU1iCjLssjJ93oMhg" found:
i:  668 url:  https://www.yelp.com/biz/tequila-time-bar-cafe-ridgewood?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
TIME TEQUILA BAR CAFE
TRIONA'S BAR RESTAURANT
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/e4FbDBoHWCOvDoHkrXzPsA ...
Result for business "e4FbDBoHWCOvDoHkrXzPsA" found:
i:  669 url:  https://www.yelp.com/biz/trionas-on-third-new-yo

Querying https://api.yelp.com/v3/businesses/Lm-N3uF0PMHuu9A7FrXYNw ...
Result for business "Lm-N3uF0PMHuu9A7FrXYNw" found:
i:  687 url:  https://www.yelp.com/biz/ca%C3%B1as-restaurant-and-bar-jamaica?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
CANAS RESTAURANT AND BAR
CASA CLEMA RESTAURANT & BAR
Querying https://api.yelp.com/v3/businesses/search ...
No businesses for CASA CLEMA RESTAURANT & BAR in New York found.
CHEAP SHOTS SPORTS BAR and LOUNGE
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/La0m0JQirzld8XSyyf37tg ...
Result for business "La0m0JQirzld8XSyyf37tg" found:
i:  689 url:  https://www.yelp.com/biz/barcelona-bar-new-york?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
CHEAP SHOTS SPORTS BAR and LOUNGE
CITI FIELD EFFEN VODKA BAR
Querying https://api.yelp.co

Querying https://api.yelp.com/v3/businesses/3JfQ4ZsJWEuIHAyVkaQ9Qg ...
Result for business "3JfQ4ZsJWEuIHAyVkaQ9Qg" found:
i:  708 url:  https://www.yelp.com/biz/landshark-bar-and-grill-times-square-new-york-4?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
LANDSHARK BAR & GRILL
LAST STOP BAR & GRILL
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/dA6hxdyzJS0xqizoJ6XztA ...
Result for business "dA6hxdyzJS0xqizoJ6XztA" found:
i:  709 url:  https://www.yelp.com/biz/last-stop-bar-and-grill-bronx?adjust_creative=pX19ggnfgJJ4FZOQklOuNA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=pX19ggnfgJJ4FZOQklOuNA
LAST STOP BAR & GRILL
LATINA'S KITCHEN & BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/Pn_KLz8cue1P0AWQgFw1Zg ...
Result for business "Pn_KLz8cue1P0AWQgFw1Zg" found:
i:  710 url:

ConnectionError: HTTPSConnectionPool(host='api.yelp.com', port=443): Max retries exceeded with url: /v3/businesses/search?term=MERMAID%2BOYSTER%2BBAR&location=New%2BYork&limit=3 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd6cf4df640>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [78]:
pd.options.display.max_columns = None


In [98]:
df.head(10)

,bar_id,bar_pub_name,borough,building,address,zip,phone,latitude,longitude,score,grade,grade_dt,yelp_image_url,yelp_url,yelp_rating,yelp_price,day_0_hours_start,day_0_hours_end,day_1_hours_start,day_1_hours_end,day_2_hours_start,day_2_hours_end,day_3_hours_start,day_3_hours_end,day_4_hours_start,day_4_hours_end,day_5_hours_start,day_5_hours_end,day_6_hours_start,day_6_hours_end
0,50041799,310 BOWERY BAR,Manhattan,310,BOWERY,10012.0,2124771166,40.724796,-73.992373,7,A,2018-05-16,https://s3-media3.fl.yelpcdn.com/bphoto/eq-6aX...,https://www.yelp.com/biz/310-bowery-bar-new-yo...,3.5,$$,1500,0200,1500,0200,1500,0200,1500,0400,1500,0400,1200,0400,1200,0200
1,41331076,55 BAR,Manhattan,55,CHRISTOPHER STREET,10014.0,2129299883,40.733712,-74.002183,17,B,2017-01-17,https://s3-media3.fl.yelpcdn.com/bphoto/0JF5nB...,https://www.yelp.com/biz/sky-55-bar-and-grill-...,4.5,$$,1130,2200,1130,2200,1130,2200,1130,2230,1130,2200,NaN,NaN,NaN,NaN
2,41420397,69 BAR LOUNGE,Queens,3821,69 STREET,11377.0,3475437183,40.746886,-73.896421,8,A,2019-10-16,https://s3-media3.fl.yelpcdn.com/bphoto/2Ms4f7...,https://www.yelp.com/biz/169-bar-new-york-4?ad...,3.5,$,1130,0400,1130,0400,1130,0400,1130,0400,1130,0400,1130,0400,1130,0400
3,50067022,A PASTA BAR,Manhattan,330,WEST BROADWAY,10013.0,6463015372,40.722107,-74.004008,5,A,2019-10-02,https://s3-media2.fl.yelpcdn.com/bphoto/4VDTeJ...,https://www.yelp.com/biz/a-pasta-bar-new-york?...,4.0,$$$,1700,2300,1700,2300,1700,2300,1200,2330,1200,2330,1200,2330,1200,2330
4,41509909,BILL'S BAR & BURGER ROCKEFELLER CENTER,Manhattan,45,ROCKEFELLER CENTER,10111.0,2127058510,40.759261,-73.977844,7,A,2019-04-29,https://s3-media3.fl.yelpcdn.com/bphoto/-lK_eG...,https://www.yelp.com/biz/bills-bar-and-burger-...,3.5,$$,0700,2200,0700,2200,0700,2200,0700,2200,0700,2300,0700,2300,0700,2200
5,50079689,BOTEQUIM 39 WINE AND BEER,Queens,2905,39TH AVE,11101.0,7184339330,40.753450,-73.934324,6,A,2019-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,40368338,BRADY'S BAR,Manhattan,1583,2 AVENUE,10028.0,2128616070,40.775434,-73.953532,13,A,2017-06-07,https://s3-media1.fl.yelpcdn.com/bphoto/o6FPsY...,https://www.yelp.com/biz/bradys-bar-new-york?a...,3.5,$,1600,0200,1600,0200,1600,0200,1600,0200,1600,0200,1200,0400,1200,0200
7,41251541,BRIDGES BAR,Bronx,4100,EAST TREMONT AVENUE,10465.0,3473989144,40.818818,-73.817114,7,A,2019-01-31,https://s3-media3.fl.yelpcdn.com/bphoto/37S0vo...,https://www.yelp.com/biz/bridges-bar-new-york?...,3.0,$$,1730,0200,1730,0200,1730,0200,1730,0200,1730,0200,1730,0200,NaN,NaN
8,41127647,BURRITO BAR,Brooklyn,307311,FLATBUSH AVENUE,NaN,7182308646,0.000000,0.000000,10,A,2019-07-18,https://s3-media4.fl.yelpcdn.com/bphoto/yLt3iA...,https://www.yelp.com/biz/burrito-bar-brooklyn-...,3.0,$$,1200,2300,1200,2300,1200,2300,1200,2300,1200,0000,1200,0000,1200,2300
9,41688632,CAMINITO SPORTS BAR AND BILLIARD,Bronx,393,EAST 198 STREET,10458.0,3475902665,40.866177,-73.886115,12,A,2016-11-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
len(df)

2464

In [107]:
len(df[~df.day_0_hours_start.isnull()])

580

In [108]:
df.to_csv('nyc_bar_crawl_w_details.csv')